# Week06b

### Example01 데이터 정제 및 시각화

In [ ]:
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def read(filename, date_idx, date_parse, year=None, bucket=7):
    days_in_year = 365
    
    freq = {}
    if year != None:
         for period in range(0, int(days_in_year / bucket)):
            freq[period] = 0
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile) #csv 파일을 열고
        next(csvreader)#첫번째 행을 건너뜀(헤더 제거)
        
        for row in csvreader:
            #빈 데이터 확인
            if row[date_idx] == '':#날짜 피드가 빈 경우 건너뜀
                continue
            
            #날짜 파싱
            t = time.strptime(row[date_idx], date_parse)#'%m/%d/%Y %H:%M:%S %p'로 데이터 파싱

            #데이터 필터링 및 집계
            if year == None:
                if not t.tm_year in freq:
                    freq[t.tm_year] = {}
                    for period in range(0, int(days_in_year / bucket)):
                        freq[t.tm_year][period] = 0
                
                if t.tm_yday < (days_in_year - 1):
                    freq[t.tm_year][int(t.tm_yday / bucket)] += 1
                    
            else:
                if t.tm_year == year and t.tm_yday < (days_in_year-1):
                    freq[int(t.tm_yday / bucket)] += 1
    
    return freq
freq = read('./311_call_center.csv', 1, '%m/%d/%Y %H:%M:%S %p', 2014)

X_train = np.asarray(list(freq.keys()))
Y_train = np.asarray(list(freq.values()))
print(freq)
plt.scatter(X_train,Y_train)
plt.show()


### Example02-a 가우시안 분포와 정규화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.0) / (2 * np.power(sig, 2.0)))

x_values = np.linspace(-3, 3, 120)
for mu, sig in [(-1,1), (0,2), (2, 3)]:
    plt.plot(x_values, gaussian(x_values, mu, sig), label=f' μ = {mu}, σ = {sig}') # μ : 뮤, σ : 시그마

plt.legend()
plt.show()

### Example02-b 가우시안 분포와 정규화

In [ ]:
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def read(filename, date_idx, date_parse, year=None, bucket=7):
    days_in_year = 365
    
    freq = {}
    if year != None:
         for period in range(0, int(days_in_year / bucket)):
            freq[period] = 0
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile) #csv 파일을 열고
        next(csvreader)#첫번째 행을 건너뜀(헤더 제거)
        
        for row in csvreader:
            #빈 데이터 확인
            if row[date_idx] == '':#날짜 피드가 빈 경우 건너뜀
                continue
            
            #날짜 파싱
            t = time.strptime(row[date_idx], date_parse)#'%m/%d/%Y %H:%M:%S %p'로 데이터 파싱

            #데이터 필터링 및 집계
            if year == None:
                if not t.tm_year in freq:
                    freq[t.tm_year] = {}
                    for period in range(0, int(days_in_year / bucket)):
                        freq[t.tm_year][period] = 0
                
                if t.tm_yday < (days_in_year - 1):
                    freq[t.tm_year][int(t.tm_yday / bucket)] += 1
                    
            else:
                if t.tm_year == year and t.tm_yday < (days_in_year-1):
                    freq[int(t.tm_yday / bucket)] += 1
    
    return freq
freq = read('./311_call_center.csv', 1, '%m/%d/%Y %H:%M:%S %p', 2014)

X_train = np.asarray(list(freq.keys()))
Y_train = np.asarray(list(freq.values()))
print(freq)
# 정규화
max_Y = np.max(Y_train)
new_Y_train = Y_train / max_Y
plt.scatter(X_train,new_Y_train)
plt.show()


### Example03 가우시안 모델 구축

In [ ]:
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def read(filename, date_idx, date_parse, year=None, bucket=7):
    days_in_year = 365
    
    freq = {}
    if year != None:
         for period in range(0, int(days_in_year / bucket)):
            freq[period] = 0
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile) #csv 파일을 열고
        next(csvreader)#첫번째 행을 건너뜀(헤더 제거)
        
        for row in csvreader:
            #빈 데이터 확인
            if row[date_idx] == '':#날짜 피드가 빈 경우 건너뜀
                continue
            
            #날짜 파싱
            t = time.strptime(row[date_idx], date_parse)#'%m/%d/%Y %H:%M:%S %p'로 데이터 파싱

            #데이터 필터링 및 집계
            if year == None:
                if not t.tm_year in freq:
                    freq[t.tm_year] = {}
                    for period in range(0, int(days_in_year / bucket)):
                        freq[t.tm_year][period] = 0
                
                if t.tm_yday < (days_in_year - 1):
                    freq[t.tm_year][int(t.tm_yday / bucket)] += 1
                    
            else:
                if t.tm_year == year and t.tm_yday < (days_in_year-1):
                    freq[int(t.tm_yday / bucket)] += 1
    
    return freq
freq = read('./311_call_center.csv', 1, '%m/%d/%Y %H:%M:%S %p', 2014)

X_train = np.asarray(list(freq.keys()))
Y_train = np.asarray(list(freq.values()))
#plt.scatter(X_train,Y_train)

maxY = np.max(Y_train)
nY_train = Y_train / maxY
#plt.scatter(X_train,nY_train)

class Model:
    def __init__(self):
        self.mu = tf.Variable(1.0, dtype=tf.float32)
        self.sig = tf.Variable(1.0, dtype=tf.float32)
    def __call__(self, x):
        x_c = tf.cast(x, tf.float32)
        return tf.exp(-tf.pow(x_c - self.mu, 2.) / (2. * tf.pow(self.sig, 2.)))

def costFunction(predicted_y, desired_y):
    loss_function = tf.square(predicted_y - desired_y)
    return tf.reduce_mean(loss_function)

learning_rate = 1.5
optimizer = tf.keras.optimizers.SGD(learning_rate)

def trainStep(model, inputs, outputs):
    with tf.GradientTape() as t:
        current_cost_function = costFunction(model(inputs), outputs)
        
    grads = t.gradient(current_cost_function, [model.mu, model.sig])
    optimizer.apply_gradients(zip(grads,[model.mu, model.sig]))
    return current_cost_function

model = Model()

training_epochs = 50
for epoch in range(training_epochs):
    for i in range(0, len(X_train)):
        _cost_function = trainStep(model, X_train[i], nY_train[i])        
    if epoch % 10 == 0:
        print("Current cost_function %f" % (_cost_function.numpy()))
     
        
mu_val = model.mu
sig_val = model.sig
print(mu_val.numpy())
print(sig_val.numpy())

plt.scatter(X_train, Y_train)
trY2 = maxY * (np.exp(-np.power(X_train - mu_val, 2.) / (2 * np.power(sig_val, 2.))))
plt.plot(X_train, trY2, 'r')
plt.show()
print("Prediction of week 35", trY2[33])
print("Actual week 35", Y_train[33])



### Example04 딥러닝 모델 코드

In [ ]:
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

def read(filename, date_idx, date_parse, year=None, bucket=7):
    days_in_year = 365
    
    freq = {}
    if year != None:
         for period in range(0, int(days_in_year / bucket)):
            freq[period] = 0
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile) #csv 파일을 열고
        next(csvreader)#첫번째 행을 건너뜀(헤더 제거)
        
        for row in csvreader:
            #빈 데이터 확인
            if row[date_idx] == '':#날짜 피드가 빈 경우 건너뜀
                continue
            
            #날짜 파싱
            t = time.strptime(row[date_idx], date_parse)#'%m/%d/%Y %H:%M:%S %p'로 데이터 파싱

            #데이터 필터링 및 집계
            if year == None:
                if not t.tm_year in freq:
                    freq[t.tm_year] = {}
                    for period in range(0, int(days_in_year / bucket)):
                        freq[t.tm_year][period] = 0
                
                if t.tm_yday < (days_in_year - 1):
                    freq[t.tm_year][int(t.tm_yday / bucket)] += 1
                    
            else:
                if t.tm_year == year and t.tm_yday < (days_in_year-1):
                    freq[int(t.tm_yday / bucket)] += 1
    
    return freq
freq = read('./311_call_center.csv', 1, '%m/%d/%Y %H:%M:%S %p', 2014)

X_train = np.asarray(list(freq.keys()))
Y_train = np.asarray(list(freq.values()))
#plt.scatter(X_train,Y_train)

model = models.Sequential([ # 모델 구축
    layers.Input(shape=(1,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),   
    layers.Dense(8, activation='relu'),   
    layers.Dense(1)
])

### start of norm on input data
from sklearn.preprocessing import MinMaxScaler # MinMaxScaler는 2차원 입력이 들어가므로 10 -> 20
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train.reshape(-1, 1))#1D->2D
Y_train = scaler.fit_transform(Y_train.reshape(-1, 1))#1D->2D

X_train = tf.convert_to_tensor(X_train,dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train,dtype=tf.float32)

learning_rate = 0.1
training_epochs = 1000
optimizer = SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
model.summary()
history = model.fit(X_train, Y_train, epochs=training_epochs, batch_size=64) # 64 Batch 사이즈에 대해 고려해보자(중요!)

y_pred = model.predict(X_train)
y_pred=scaler.inverse_transform(y_pred) #원래의 크기로 복구
y_pred[y_pred <0] =0 #음수값이면 0
X_train = X_train.numpy()*52 #원래의 크기로 복구
Y_train = scaler.inverse_transform(Y_train)

plt.scatter(X_train, Y_train, color='blue', s=5,label='Actual Data')
plt.scatter(X_train, y_pred, color='red', s=5,label='Predicted Data')
plt.legend()
plt.show()

#plt.plot(history.history[ 'loss'])
#print(history.history['loss']) 
#print(history.history['accuracy'])
#### 매개변수 값을 확인하는 방법
parameters = model.get_weights()
######